# Performance of quantization

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import pylab
import math
import numpy as np
from scipy import signal
import cv2
import os

In [ ]:
def quantizer(x, quantization_step):
    k = (x / quantization_step).astype(np.int16)
    return k

def dequantizer(k, quantization_step):
    y = quantization_step * k
    return y

def q_deq(x, quantization_step):
    k = quantizer(x, quantization_step)
    y = dequantizer(k, quantization_step)
    return k, y

# Notice that, although this is a dead-zone quantizer, we are not going
# to work with negative samples, and therefore, the dead-zone
# does not have any effect.

In [ ]:
def load_frame(prefix):
    fn = f"{prefix}.png"
    frame = cv2.imread(fn, cv2.IMREAD_UNCHANGED) # [rows, cols, comp]
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = np.array(frame)
    frame = frame.astype(np.float32) - 32768.0
    frame = frame.astype(np.uint8)
    return frame

def write_frame(frame, prefix):
    frame = frame.astype(np.float32)
    frame += 32768.0
    frame = frame.astype(np.uint16)
    cv2.imwrite(f"{prefix}.png", frame)

In [ ]:
def load_indexes(prefix):
    load_frame(prefix)
    
def write_indexes(prefix):
    write_frame(prefix)

In [ ]:
fn = "/home/vruiz/MRVC/sequences/stockholm/000"
frame = load_frame(fn)
print(frame.max(), frame.min())

In [ ]:
def normalize(img):
    max_component = np.max(img)
    min_component = np.min(img)
    max_min_component = max_component - min_component
    return (img-min_component)/max_min_component

def show_frame(frame, prefix):
    frame = normalize(frame)
    plt.figure(figsize=(10,10))
    plt.title(prefix, fontsize=20)
    plt.imshow(frame)

In [ ]:
show_frame(frame, fn)

In [ ]:
def average_energy(x):
    return np.sum(x.astype(np.double)*x.astype(np.double))/len(x)

def RMSE(x, y):
    error_signal = x - y
    return math.sqrt(average_energy(error_signal))

In [ ]:
def byte_rate(frame):
    write_frame(frame, "/tmp/frame")
    length_in_bytes = os.path.getsize("/tmp/frame.png")
    return length_in_bytes

def entropy_in_bits_per_symbol(sequence_of_symbols):
    value, counts = np.unique(sequence_of_symbols, return_counts = True)
    probs = counts / len(sequence_of_symbols)
    n_classes = np.count_nonzero(probs)

    if n_classes <= 1:
        return 0

    entropy = 0.
    for i in probs:
        entropy -= i * math.log(i, 2)

    return entropy

In [ ]:
def RD_curve(x):
    points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        k, y = q_deq(x, 1<<q_step)
        rate = byte_rate(k)
        distortion = RMSE(x, y)
        points.append((rate, distortion))
    return points

RD_points = RD_curve(frame)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points), c='r', marker="o")
pylab.title("RD Only Quantization")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("RMSE")
pylab.show()

In [ ]:
k, y = q_deq(frame, 64)
show_frame(y, "")